In [1]:
import time
import spacy
import lftk
import pandas as pd
from datasets import load_dataset

In [2]:
# load a trained pipeline of your choice from spacy
nlp = spacy.load("en_core_web_sm")

# readability formula scores - we can always add more general linguistic features
metrics = lftk.search_features(family="readformula", return_format = "list_key")

In [3]:
# data: generated summaries, gold: human-written summaries
data = pd.read_csv('../output/baseline_test.csv', usecols=['summary_generated'])
gold = load_dataset("FiscalNote/billsum")["test"].to_pandas()

In [4]:
# returns a dictionary of readability scores from LFTK
def readability_score_single(text):
    doc = nlp(text)
    LFTK = lftk.Extractor(docs = doc)
    features = LFTK.extract(features = metrics)
    return features

def readability_eval(df, column):
    start_time = time.time()
    df["scores"] = df[column].apply(readability_score_single)
    print(f"{time.time() - start_time} sec to run")
    return

In [5]:
# replaces the dictionary column with exploded keys as new columns
def convert_cols(df, column):
    return df.join(
        pd.json_normalize(
            df[column], record_prefix="gold_"
        )
    ).drop(
        column,
        axis='columns'
    )

In [6]:
readability_eval(data, "summary_generated")
readability_eval(gold, "summary")

81.62794089317322 sec to run
97.93178701400757 sec to run


In [7]:
gen = convert_cols(data, "scores")
gold = convert_cols(gold, "scores")

In [20]:
# gold.describe().to_csv("gold_describe.csv")
gold.describe().compare(gen.describe(), keep_equal=True, align_axis=1, result_names=("gold", "generated"))

# aggr.to_csv("baseline_gold_compare.csv", index=False)

fkre                    fkgl                   fogi              \
            gold   generated        gold  generated        gold   generated   
mean   12.950785    5.292479   22.548614  26.011575   28.701746   32.256271   
std    28.561424   39.147077   10.030598  14.147657   10.336453   14.545976   
min  -333.371000 -197.615000    6.311000   6.217000   10.150000    8.969000   
25%     2.670000   -8.085000   16.792000  17.666000   22.815000   23.667000   
50%    17.009000   12.846000   20.577000  22.472000   26.733000   28.619000   
75%    29.321000   29.003000   25.451000  29.742000   31.892000   36.000000   
max    80.098000   81.956000  154.161000  97.404000  163.263000  105.737000   

           smog                  cole                   auto              
           gold  generated       gold  generated        gold   generated  
mean  11.587442  12.407292  15.162678  14.809797   26.354612   30.819341  
std    3.764673   4.721686   2.216489   2.464433   12.738404   17.997384  
min    0.000000   0.000000   7.802000   6.261000    5.720000    6.407000  
25%    9.329000   9.329000  13.660000  13.118000   19.089000   20.248000  
50%   11.220000  11.892000  15.045000  14.684000   23.714000   26.103000  
75%   13.398000  14.750000  16.556000  16.320000   29.975000   35.103000  
max   35.676000  36.579000  25.251000  24.759000  194.066000  122.723000

In [15]:
# gen.describe().to_csv("gen_describe.csv")
gen.describe()

,fkre,fkgl,fogi,smog,cole,auto
count,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000
mean,5.292479,26.011575,32.256271,12.407292,14.809797,30.819341
std,39.147077,14.147657,14.545976,4.721686,2.464433,17.997384
min,-197.615000,6.217000,8.969000,0.000000,6.261000,6.407000
25%,-8.085000,17.666000,23.667000,9.329000,13.118000,20.248000
50%,12.846000,22.472000,28.619000,11.892000,14.684000,26.103000
75%,29.003000,29.742000,36.000000,14.750000,16.320000,35.103000
max,81.956000,97.404000,105.737000,36.579000,24.759000,122.723000
